In [1]:
import numpy as np
import networkx as nx
import pickle as pk
import torch
import matplotlib.pyplot as plt
from stellargraph.data import UnsupervisedSampler
from tqdm import tqdm

from stellargraph import StellarGraph
from stellargraph.mapper import GraphSAGENodeGenerator,GraphSAGELinkGenerator
from stellargraph.data import EdgeSplitter
from stellargraph.layer import GraphSAGE, HinSAGE, link_classification

from tensorflow import keras
from sklearn import preprocessing, feature_extraction, model_selection

from stellargraph import globalvar
from stellargraph import datasets
from IPython.display import display, HTML

import logging
import time
from collections import defaultdict
import os
import scipy.sparse as sp
import torch
from torch.nn import functional as F

datasetname = 'MV-APIMethod'#有数据集SH_S、SH_L、MV,SH_L-APIMethod
threshold1 = 0.5
threshold2 = 0.5
batch_size = 1024
epochs = 100
num_samples = [20, 10]
layer_sizes = [64, 64]

best_suc = [0]*21
best_pre = [0]*21
best_recall = [0]*21
pro_best_suc = [0]*21
pro_best_pre = [0]*21
pro_best_recall = [0]*21
test_config = 'C2.1'

def getg(data):
    g = nx.Graph()
    n = len(data.item_method_id)
    # 顶点
    point = []
    for i in range(n):
        point.append(i)
    g.add_nodes_from(point)
    # 边权重
    edglist = []
    edges = set()
    for user, items in tqdm(data.invocation_mx.items()):
        for i in range(len(items)):
            for j in range(i+1,len(items)):
                edges.add((items[i],items[j]))
    
    for edg in tqdm(edges):
        edglist.append((edg[0],edg[1],float(data.adj[edg[0],edg[1]])))
        #edglist.append((edg[0],edg[1]))

    g.add_weighted_edges_from(edglist)
    #g.add_edges_from(edglist)
    return g

def to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

def build_my_new_adj_matrix(data,train_dict):
    n = len(data.item_method_id)
    A = sp.dok_matrix((n, n), dtype=np.float32)
    FD = defaultdict(int)
    for user, items in tqdm(train_dict.items()):
        for i in range(len(items)):
            FD[items[i]] += 1
            for j in range(i+1,len(items)):
                if A[items[i],items[j]] == 0:
                    A[items[i],items[j]] = A[items[j],items[i]] = len(data.invocation_mx.items())
                else:
                    A[items[i], items[j]] = A[items[j], items[i]] = A[items[j], items[i]] + len(data.invocation_mx.items())
    print('build_my_new_adj_matrix finish')
    data.FD = FD
    data.adj = to_torch_sparse_tensor(A)
    

def load_mydata(dataset_name):
    name = './tmp/%s-mydata.pk' % dataset_name
    if not os.path.exists(name):
        print('no file.')
    with open(name, 'rb') as f:
        data = pk.load(f)
        print('load dataset from disk.')
    #data.adj = to_torch_sparse_tensor(data.adj)
    return data

def get_node_embeddings(data):
    g = getg(data)
    g_feature_attr = g.copy()
    for node_id, node_data in g_feature_attr.nodes(data=True):
        node_data["feature"] = data.item_pre_emb[node_id].numpy()

    G = StellarGraph.from_networkx(
        g_feature_attr, node_features="feature", node_type_default="API", edge_type_default="Attribute"
    )
    edge_splitter_test = EdgeSplitter(G)

    # Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G, and obtain the
    # reduced graph G_test with the sampled links removed:
    G_test, edge_ids_test, edge_labels_test = edge_splitter_test.train_test_split(
        p=0.1, method="global", keep_connected=True
    )
    # Define an edge splitter on the reduced graph G_test:
    edge_splitter_train = EdgeSplitter(G_test)

    # Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G_test, and obtain the
    # reduced graph G_train with the sampled links removed:
    G_train, edge_ids_train, edge_labels_train = edge_splitter_train.train_test_split(
        p=0.1, method="global", keep_connected=True
    )

    train_gen = GraphSAGELinkGenerator(G_train, batch_size, num_samples)
    train_flow = train_gen.flow(edge_ids_train, edge_labels_train, shuffle=True)
    test_gen = GraphSAGELinkGenerator(G_test, batch_size, num_samples)
    test_flow = test_gen.flow(edge_ids_test, edge_labels_test)
    graphsage = GraphSAGE(
        layer_sizes=layer_sizes, generator=train_gen, bias=True, dropout=0.3
    )
    x_inp, x_out = graphsage.in_out_tensors()
    prediction = link_classification(
        output_dim=1, output_act="relu", edge_embedding_method="ip"
    )(x_out)
    model = keras.Model(inputs=x_inp, outputs=prediction)

    model.compile(
        optimizer=keras.optimizers.Adam(lr=1e-3),
        loss=keras.losses.binary_crossentropy,
        metrics=[keras.metrics.binary_accuracy],
    )
    history = model.fit(train_flow, epochs=epochs, validation_data=test_flow, verbose=2)
    x_inp_src = x_inp[0::2]
    x_out_src = x_out[0]
    embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)
    node_ids = G_train.nodes()
    node_gen = GraphSAGENodeGenerator(G_train, batch_size, num_samples).flow(node_ids)
    node_embeddings = embedding_model.predict(node_gen, workers=4, verbose=1)
    return node_embeddings

#threshold1取值（0，1）表示考虑节点相似特征的阈值，值越大候选特征节点越少
#threshold2取值（0，1）表示节点属性特征的重要性，越小越不重要
def build_new_relation(ratings,threshold1 = 0.8,threshold2 = 0.001):
    dataset = load_mydata(datasetname)
    n = len(ratings)
    for i in tqdm(range(n)):
        for j in range(i+1,n):
            simily = float(ratings[i][j])
            if simily >= threshold1:
                dataset.adj[i,j] = threshold2*simily
    return dataset
    
def get_my_top_items(tensor):
    item_dict = {}
    for i in tqdm(range(len(tensor))):
        if tensor[i].item() !=0 :
            item_dict[i] = tensor[i].item()
    #print('get_my_top_items==>item_dict',item_dict)
    top_items = [item[0] for item in sorted(item_dict.items(),key=lambda item:item[1],reverse=True)]
    #print('get_my_top_items==>top_items',top_items)
    return top_items[:21]

def get_my_top_items2(data,adj,Q,ratings,a,b):
    #链路预测
    diag = torch.diag(ratings) #获取对角为一维向量
    diag_embed = torch.diag_embed(diag)  # 由diag恢复为对角矩阵
    link_embed = ratings - diag_embed
    rowsoftmax = F.softmax(link_embed,dim=1)
    link_q1 = torch.zeros(size=[len(rowsoftmax[0])])
    for q in Q:
        link_q1 = link_q1 + rowsoftmax[q]
        
    #贝叶斯预测
    M = len(data.invocation_mx.items())
    D = set()
    FD = data.FD
    
    for q in Q:
        tensor = adj[q]
        for i in range(len(tensor)):
            if tensor[i].item() != 0:
                D.add(i)

    link_q2 = torch.zeros(size=[len(adj)])
    print(len(link_q2))
    for d in D:
        fd = FD[d]
        fdq = 1
        for q in Q:
            tensor = adj[q]
            fdq = fdq*(tensor[d].item()*1.0/fd)
        #利用贝叶斯求得d被预测的概率
        p2 = fdq*fd*1.0/M
        link_q2[d] = p2
    

    link_q = link_q1*a+link_q2*b
    arr,top_items = torch.sort(link_q,descending=True)
    top_items = top_items[:21]
    #top_items = [item[0] for item in sorted(item_dict.items(), key=lambda item: item[1], reverse=True)]
    # print('get_my_top_items==>top_items',top_items)
    return top_items.numpy()

def myeval(dataset,ratings,a,b):
    test_set = dataset.test_dict
    logger.info('test start. test set size: %d' % len(test_set))
    t1 = time.time()
    users = np.asarray(list(test_set.keys()))  # 训练集的方法编号数组

    top_items = []
    used_items = []

    for userid in tqdm(users):
        used_items.append(set(dataset.train_dict[userid]))
        #print(dataset.train_dict[userid],dataset.train_dict[userid][0])
        #top_items.append(get_my_top_items(dataset.adj[dataset.train_dict[userid][0]]))
        top_items.append(get_my_top_items2(dataset,dataset.adj,dataset.train_dict[userid],ratings,a,b))

    #print('myeval2=>top_items',top_items)
    #print('myeval2=>used_items', used_items)

    items = []
    for i, item in enumerate(top_items):  # 第i个测试方法推荐的API列表item
        # if i<=20:
        rec_item = [tid for tid in item if tid not in used_items[i]]
        # print(rec_item)
        items.append(rec_item[:20])

    def getMAP(N):
        qarr = []
        for i, uid in enumerate(users):
            r = 0
            drarr = []
            for k in range(1, N+1):
                intersect = set(items[i][:k]) & set(test_set[uid])
                p = len(intersect) / k
                newr = len(intersect) / len(set(test_set[uid]))
                dr = (newr-r)*p
                drarr.append(dr)
                r = newr
            qarr.append(np.sum(drarr))
        return np.sum(qarr)/len(qarr)
     
    def res_at_k(k):
        suc_methods = []
        precisions = []
        recalls = []
        proj_suc = defaultdict(list)
        proj_pre = defaultdict(list)
        proj_recall = defaultdict(list)

        for i, uid in enumerate(users):
            pid = dataset.test_user2proj[uid]
            intersect = set(items[i][:k]) & set(test_set[uid])
            if len(intersect) > 0:
                suc_methods.append(uid)
                proj_suc[pid].append(1)
            else:
                logger.debug('failed uid %d' % uid)
                logger.debug('GT:{}, REC:{}'.format(test_set[uid], items[i]))
                proj_suc[pid].append(0)
            p = len(intersect) / k
            r = len(intersect) / len(set(test_set[uid]))
            precisions.append(p)
            recalls.append(r)
            proj_pre[pid].append(p)
            proj_recall[pid].append(r)
        suc_rate = len(suc_methods) / len(users)
        logger.info('----------------------result@%d--------------------------' % k)
        logger.info('success rate at method level %f' % (suc_rate))
        logger.info('mean precision:%f, mean recall:%f' % (np.mean(precisions), np.mean(recalls)))

        suc_project = [np.mean(val) for val in proj_suc.values()]
        pres = [np.mean(val) for val in proj_pre.values()]
        recs = [np.mean(val) for val in proj_recall.values()]
        logger.info('**********************************************************')
        logger.info('success rate at project level %f' % (np.mean(np.mean(suc_project))))
        logger.info('mean precision:%f, mean recall:%f' % (np.mean(pres), np.mean(recs)))
        return suc_rate, np.mean(precisions), np.mean(recalls),np.mean(np.mean(suc_project)),np.mean(pres), np.mean(recs)

    t2 = time.time()
    logger.info('test end time: {}s'.format(t2 - t1))
    for i in range(1, 21):
        suc, pre, rec, pro_suc, pro_pre, pro_rec = res_at_k(i)
        if suc > best_suc[i]:
            best_suc[i] = suc
        if pre > best_pre[i]:
            best_pre[i] = pre
        if rec > best_recall[i]:
            best_recall[i] = rec
        logger.warning('method level => top %d : best suc %f, best pre %f,  best recall %f' % (i, best_suc[i], best_pre[i], best_recall[i]))

        if pro_suc > pro_best_suc[i]:
            pro_best_suc[i] = pro_suc
        if pro_pre > pro_best_pre[i]:
            pro_best_pre[i] = pro_pre
        if pro_rec > pro_best_recall[i]:
            pro_best_recall[i] = pro_rec
        logger.warning('project level => top %d : best suc %f, best pre %f,  best recall %f' % (i, pro_best_suc[i], pro_best_pre[i], pro_best_recall[i]))

    logger.info('MAP: %f' % (getMAP(20)))

In [2]:
#载入数据并划分数据集
data = load_mydata(datasetname)
data.split_data(test_config)
build_my_new_adj_matrix(data,data.train_dict)

  0%|          | 32/94530 [00:00<13:20, 118.00it/s]

load dataset from disk.
total user methods:94530, test_proj:{513, 1538, 1027, 516, 5, 1539, 4, 520, 9, 519, 1032, 524, 1541, 526, 14, 2, 17, 18, 19, 530, 1557, 22, 535, 536, 1050, 27, 1564, 1565, 542, 31, 1052, 1055, 34, 1570, 1, 1566, 1062, 553, 554, 45, 1073, 1074, 1586, 563, 53, 54, 1587, 567, 1593, 1594, 1079, 565, 1597, 572, 1599, 574, 66, 579, 580, 1092, 582, 583, 587, 76, 1100, 80, 1105, 82, 1108, 85, 1110, 89, 602, 96, 1122, 1126, 1129, 107, 620, 1133, 108, 113, 1142, 632, 122, 1147, 638, 642, 1154, 133, 1157, 646, 648, 1161, 141, 654, 655, 142, 1169, 1170, 656, 148, 1173, 152, 1178, 1179, 1180, 667, 1182, 670, 1183, 161, 159, 1187, 1188, 672, 1190, 678, 170, 171, 1194, 1197, 686, 175, 687, 177, 178, 692, 693, 182, 187, 1213, 191, 192, 704, 703, 1217, 712, 713, 1224, 202, 1228, 204, 1230, 716, 722, 211, 1236, 1238, 1239, 219, 733, 223, 737, 227, 740, 229, 1253, 1255, 744, 743, 233, 746, 1260, 752, 1265, 240, 1269, 1271, 760, 249, 1274, 762, 253, 254, 1277, 768, 1283, 259, 260, 

  0%|          | 20/94530 [00:00<07:55, 198.80it/s]

train set methods count:94530, invocation: 5280371


100%|██████████| 94530/94530 [06:57<00:00, 226.44it/s] 


build_my_new_adj_matrix finish


In [3]:
node_embeddings = get_node_embeddings(data)

 68%|██████▊   | 703392/1028787 [2:49:21<1:15:10, 72.14it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 75%|███████▍  | 766832/1028787 [3:04:17<1:03:30, 68.74it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 80%|████████  | 825650/1028787 [3:18:13<44:36, 75.91it/s]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config v

156/156 - 907s - loss: 0.6287 - binary_accuracy: 0.6331 - val_loss: 0.8162 - val_binary_accuracy: 0.6098
Epoch 4/100
156/156 - 896s - loss: 0.6431 - binary_accuracy: 0.6299 - val_loss: 0.8605 - val_binary_accuracy: 0.5922
Epoch 5/100
156/156 - 913s - loss: 0.6091 - binary_accuracy: 0.6570 - val_loss: 0.7219 - val_binary_accuracy: 0.6359
Epoch 6/100
156/156 - 945s - loss: 0.6056 - binary_accuracy: 0.6639 - val_loss: 0.7402 - val_binary_accuracy: 0.6249
Epoch 7/100
156/156 - 936s - loss: 0.5965 - binary_accuracy: 0.6725 - val_loss: 0.6996 - val_binary_accuracy: 0.6481
Epoch 8/100
156/156 - 906s - loss: 0.6734 - binary_accuracy: 0.6406 - val_loss: 1.0854 - val_binary_accuracy: 0.5345
Epoch 9/100
156/156 - 909s - loss: 0.6327 - binary_accuracy: 0.6327 - val_loss: 0.8254 - val_binary_accuracy: 0.6032
Epoch 10/100
156/156 - 910s - loss: 0.6076 - binary_accuracy: 0.6611 - val_loss: 0.7557 - val_binary_accuracy: 0.6252
Epoch 11/100
156/156 - 908s - loss: 0.5946 - binary_accuracy: 0.6755 - val_

Epoch 73/100
156/156 - 955s - loss: 0.6039 - binary_accuracy: 0.6653 - val_loss: 0.7894 - val_binary_accuracy: 0.6024
Epoch 74/100
156/156 - 929s - loss: 0.6005 - binary_accuracy: 0.6691 - val_loss: 0.7557 - val_binary_accuracy: 0.6159
Epoch 75/100
156/156 - 917s - loss: 0.5963 - binary_accuracy: 0.6746 - val_loss: 0.7961 - val_binary_accuracy: 0.6058
Epoch 76/100
156/156 - 897s - loss: 0.5940 - binary_accuracy: 0.6750 - val_loss: 0.7170 - val_binary_accuracy: 0.6335
Epoch 77/100
156/156 - 910s - loss: 0.5875 - binary_accuracy: 0.6823 - val_loss: 0.7072 - val_binary_accuracy: 0.6364
Epoch 78/100
156/156 - 921s - loss: 0.5858 - binary_accuracy: 0.6833 - val_loss: 0.7018 - val_binary_accuracy: 0.6393
Epoch 79/100
156/156 - 923s - loss: 0.7785 - binary_accuracy: 0.5709 - val_loss: 0.8163 - val_binary_accuracy: 0.5942
Epoch 80/100
156/156 - 900s - loss: 0.5985 - binary_accuracy: 0.6696 - val_loss: 0.7886 - val_binary_accuracy: 0.6037
Epoch 81/100
156/156 - 904s - loss: 0.5913 - binary_accu

In [4]:
#得到节点嵌入和链接相似性
api_embeddings = torch.from_numpy(node_embeddings)
ratings = api_embeddings.mm(api_embeddings.transpose(0, 1))

In [5]:
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time())) 

logging.basicConfig(format='%(asctime)s-%(levelname)s:%(message)s',
                    filename='./log/GLAPI-GraphSAGE-noPro_'+datasetname+'_'+str(threshold1)+'_'+str(threshold2)+'_'+str(batch_size)+'_'+str(epochs)+'_'+now+'.log',
                    filemode='a', level=logging.INFO)
logger = logging.getLogger(__name__)

In [6]:
t1 = time.time()
myeval(data,ratings,threshold1,threshold2)
t2 = time.time()
logger.warning('predition time : %d' % (t2-t1))

  0%|          | 0/4064 [00:00<?, ?it/s]

30442


  0%|          | 1/4064 [00:10<12:08:13, 10.75s/it]

30442


  0%|          | 2/4064 [00:33<20:14:18, 17.94s/it]

...

100%|█████████▉| 4062/4064 [56:51:16<00:19,  9.95s/it]

30442


100%|█████████▉| 4063/4064 [56:53:28<00:46, 46.50s/it]

30442


100%|██████████| 4064/4064 [56:54:14<00:00, 50.41s/it]
